In [221]:
import import_ipynb
import itertools

import random as rnd
import Cloud as cld
import numpy as np
import copy

from tqdm.notebook import tqdm

In [295]:
def shuffle(lst):
    arr = lst
    # Get length of List
    n = len(arr)
 
    #repeat the following for n number of times
    for i in range(n):
        #select an index randomly
        j = rnd.randint(0, n-1)
        #delete the element at that index.
        element=arr.pop(j)
        #now append that deleted element to the list
        arr.append(element)
    return arr

In [156]:
class Chromosome:
    def __init__(self, servers, processes, fitness, individual=None, generation=0):
        self.servers = servers
        self.processes = processes
        self.fitness = fitness
        self.average_fitness = sum([y for x, y in self.fitness])/len(self.fitness)
        self.individual = individual
        self.generation = generation
    def __lt__(self, other):
         return self.average_fitness < other.average_fitness
    def __repr__(self):
        return "Average Chromosome fitness: {0}\n\tIndividual: {1}\n\tGENERATION: {2}".format(self.average_fitness, self.individual, self.generation)

In [157]:
num_servers = 20
cores = [16, 18, 20, 24, 48, 96]
cpu_speeds = [3, 4, 5, 10, 20]
rnd.seed(42)
servers = [cld.Server(name=x, cpu_speed=cpu_speeds[rnd.randint(0, 4)], cores=cores[rnd.randint(0, 4)]) for x in range(0, num_servers)]

In [174]:
num_processes = 100
processes_lengths = [round(rnd.uniform(0.02, 1.000), 2) for x in range(0, num_processes)] 
# Fraction of billion of instructions. 1 = 1 billion instructions
processes = [cld.Process(x, processes_lengths[rnd.randint(0, len(processes_lengths)-1)]) for x in range(0, num_processes)]

In [159]:
# Must split the processes each time for each chromosome!!!

In [160]:
split_processes = np.array_split(np.asarray(processes), len(servers))

#### Fitness function to calculate the fitness score of each gene
Calculated the max time that each server expends on its own workload.<br>
Need to take into account the speed of the processors

In [146]:
def server_fitness(server):
    cores, speed, processes = server.cores, server.cpu_speed, server.workload
    print(processes)
    fitness = sorted([process.process_length for process in processes[:cores]])
    for process in processes[cores:]:
        fitness[0] += process.process_length
        fitness = sorted(fitness)
    return fitness

In [163]:
def fitness(chromosome):
    servers_fitness = []
    for server in (chromosome):
        cores, speed, processes = server.cores, server.cpu_speed, server.workload
        fitness = sorted([process.process_length for process in processes[:cores]])
        for process in processes[cores:]:
            fitness[0] += process.process_length
            fitness = sorted(fitness)
        servers_fitness.append((server.name, fitness[-1] / speed))
    return servers_fitness

In [164]:
def overall_fitness(population):
    overall_fitnesses = []
    solution = 0
    for chromosome in population:
        overall_fitnesses.append((fitness(chromosome), solution))
        solution += 1
    return overall_fitnesses

#### Function to create a first generation chromosome

In [296]:
def make_first_chromosome(servers, processes, individual):
    shuffled_processes = shuffle(processes)#np.random.choice(copy.deepcopy(processes[:]), len(processes), replace=False)#copy.deepcopy(processes[:])
    #np.random.shuffle(shuffled_processes)
    split_processes = np.array_split(np.asarray(shuffled_processes), len(servers))
    for i in range(len(split_processes)):
        servers[i].workload = split_processes[i]
    return Chromosome(servers, processes, fitness(servers), individual)#(servers, fitness(servers))#, split_processes)

In [297]:
chromo_1 = make_first_chromosome(servers, processes, 0)

In [298]:
chromo_2 = make_first_chromosome(servers, processes, 0)

In [299]:
chromo_1.servers[0].workload, chromo_2.servers[0].workload

(array([PID: 2
        	Length of process: 0.44 instructions,
        PID: 19
        	Length of process: 0.45 instructions,
        PID: 24
        	Length of process: 0.72 instructions,
        PID: 28
        	Length of process: 0.89 instructions,
        PID: 29
        	Length of process: 0.86 instructions], dtype=object),
 array([PID: 2
        	Length of process: 0.44 instructions,
        PID: 19
        	Length of process: 0.45 instructions,
        PID: 24
        	Length of process: 0.72 instructions,
        PID: 28
        	Length of process: 0.89 instructions,
        PID: 29
        	Length of process: 0.86 instructions], dtype=object))

In [270]:
#make_first_chromosome(servers, processes, "A")

In [271]:
import copy
a = [1,2,3]
type(copy.deepcopy(a))

list

In [272]:
a = np.random.choice(a, len(a), replace=False)
a

array([2, 3, 1])

#### Function to create the first generation population

In [305]:
def make_first_gen(servers, processes, pop_size):
    population = []
    x = 0
    for i in tqdm(range(pop_size)):
        rnd.seed(i)#RandomState(i)
        #np.random.shuffle(processes)
        population.append(make_first_chromosome(servers, rnd.sample(copy.deepcopy(processes[:]), len(processes)), x))
        x += 1
    return sorted(population)

In [306]:
first_gen = make_first_gen(servers, processes, num_servers)

  0%|          | 0/20 [00:00<?, ?it/s]

In [307]:
a = [x.pid for x in first_gen[0].servers[0].workload]
b = [x.pid for x in first_gen[1].servers[0].workload]
d = [x.pid for x in first_gen[2].servers[0].workload]

In [308]:
a == b == d

True

In [309]:
for i in range(len(a)):
    print(a[i], " -- ", b[i], " -- ", d[i])

73  --  73  --  73
98  --  98  --  98
20  --  20  --  20
1  --  1  --  1
66  --  66  --  66


In [229]:
first_gen[0].fitness, first_gen[1].fitness

([(0, 0.20666666666666667),
  (1, 0.2),
  (2, 0.22),
  (3, 0.3333333333333333),
  (4, 0.25),
  (5, 0.088),
  (6, 0.25),
  (7, 0.23),
  (8, 0.048),
  (9, 0.16666666666666666),
  (10, 0.1475),
  (11, 0.093),
  (12, 0.053000000000000005),
  (13, 0.192),
  (14, 0.2475),
  (15, 0.198),
  (16, 0.2425),
  (17, 0.11200000000000002),
  (18, 0.3333333333333333),
  (19, 0.33)],
 [(0, 0.2966666666666667),
  (1, 0.198),
  (2, 0.1475),
  (3, 0.3233333333333333),
  (4, 0.16666666666666666),
  (5, 0.072),
  (6, 0.13666666666666666),
  (7, 0.2475),
  (8, 0.043),
  (9, 0.3333333333333333),
  (10, 0.22),
  (11, 0.091),
  (12, 0.096),
  (13, 0.184),
  (14, 0.2475),
  (15, 0.16799999999999998),
  (16, 0.25),
  (17, 0.2),
  (18, 0.31),
  (19, 0.27999999999999997)])

#### Mutation Function
This function remixes the process list in a chromosome

In [246]:
def mutation(chromosome):
    pass

#### Crossover Function
This function selects the best subjects from one generation and "breeds" them into the next. <br>
The two best subjects from the previous generation cross into the next. <br>
<br>
##### Crossbreed Logic
Compare the best performing workloads on each server. <br>
Take workload with best performance. <br>
If there are shared processess between servers:
1. Keep process on faster server;
2. Eliminate the process on slower server;
3. Place this process in a list;
4. Insert a random pending process from the list; **This is a mutation** 
5. Repeat redistribution 3. and 4. for num_children times.
6. The parents cross in the next generation pool.<br>

This should ensure a steadily improve each generation.

In [249]:
def crossover(generation, num_children, num_surv=2):
    # Select best specimen
    survivors = generation[:2]#[:num_surv]
    # Num_surv is variable but let's assume it is always two
    father, mother = survivors[0], survivors[1]
    # Here are placed the best servers with no workload in common
    absolute_gene_expression = []
    for i in range(len(father.workload)):
        
    # Check fittest servers between the two chromosomes
        # Need to calculate the fitness of each server

In [248]:
first_gen[:4]

[Average Chromosome fitness: 8.927691666666664
 	Individual: 3
 	GENERATION: 0,
 Average Chromosome fitness: 8.942316666666667
 	Individual: 4
 	GENERATION: 0,
 Average Chromosome fitness: 8.942441666666666
 	Individual: 1
 	GENERATION: 0,
 Average Chromosome fitness: 8.943133333333332
 	Individual: 5
 	GENERATION: 0]

In [213]:
a, b = first_gen[0], first_gen[4]
server_fitness(a.__dict__["servers"][0]) == server_fitness(b.__dict__["servers"][0])#.workload

[PID: 23937
 	Length of process: 0.3 instructions
 PID: 18078
 	Length of process: 0.17 instructions
 PID: 26913
 	Length of process: 0.23 instructions ...
 PID: 12210
 	Length of process: 0.73 instructions
 PID: 4394
 	Length of process: 0.24 instructions
 PID: 12619
 	Length of process: 0.47 instructions]
[PID: 23937
 	Length of process: 0.3 instructions
 PID: 18078
 	Length of process: 0.17 instructions
 PID: 26913
 	Length of process: 0.23 instructions ...
 PID: 12210
 	Length of process: 0.73 instructions
 PID: 4394
 	Length of process: 0.24 instructions
 PID: 12619
 	Length of process: 0.47 instructions]


True

array([PID: 23937
       	Length of process: 0.3 instructions,
       PID: 18078
       	Length of process: 0.17 instructions,
       PID: 26913
       	Length of process: 0.23 instructions, ...,
       PID: 12210
       	Length of process: 0.73 instructions,
       PID: 4394
       	Length of process: 0.24 instructions,
       PID: 12619
       	Length of process: 0.47 instructions], dtype=object)

In [216]:
a.__dict__["servers"][0].workload==b.__dict__["servers"][0].workload

array([ True,  True,  True, ...,  True,  True,  True])